---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [24]:
def date_sorter():
    import re
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    global df
    data = []
    data = pd.Series(data)
    for i in range(0, len(df)) :
        data[i] = df[i].rstrip()
    ##data = [w for w in data if re.extract('[0-9]+/[0-9]+/[0-9]+', w)]
    data = pd.DataFrame(data = data).rename(columns = {0:'text'})
    date = data['text'].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/-](?P<day>\d?\d)[/-](?P<year>\d{4}))')
    index_left = ~data.index.isin([x[0] for x in date.index])
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/-](?P<day>\d?\d)[/-](?P<year>\d{2}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<month>(?:Jan|Feb|Mar|Apr|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]*)[- ]*(?P<day>\d?\d)[-, ]*(?P<year>\d{4}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<day>\d?\d)[ ](?P<month>(?:Jan|Feb|Mar|Apr|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]*)[, ]*(?P<year>\d{4}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<month>(?:Jan|Feb|Mar|Apr|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]*)[ ](?P<day>(?:\d?\d)[a-z]{2})[,](?P<year>\d{4}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##Feb 2009; Sep 2009; Oct 2010
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<month>(?:Jan|Feb|Mar|Apr|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.]*)[ ](?P<year>\d{4}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##6/2008; 12/2009
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/](?P<year>\d{4}))'))
    index_left = ~data.index.isin([x[0] for x in date.index])
    ##2009; 2010
    date = date.append(data[index_left]['text'].str.extractall(r'(?P<origin>(?P<year>\d{4}))'))
    date.reset_index(inplace = True)
    date.drop(date[date.match > 0].index, inplace=True)
    date['day'] = date['day'].fillna('01')
    date['month'] = date['month'].fillna('01')
    date['year']=date['year'].apply(str)
    date['year']=date['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
    date.drop('match', axis = 1, inplace=True)
    date['month']=date['month'].apply(str)
    date['month']=date['month'].apply(lambda x: '0'+x if len(x)<=1 else x)
    date['month']=date['month'].apply(lambda x: '01' if x in ['Jan', 'January', 'Janaury', 'Jan.', 'January.'] else x)
    date['month']=date['month'].apply(lambda x: '02' if x in ['Feb', 'February', 'Feb.', 'February.'] else x)
    date['month']=date['month'].apply(lambda x: '03' if x in ['Mar', 'March', 'Mar.', 'March.'] else x)
    date['month']=date['month'].apply(lambda x: '04' if x in ['Apr', 'April', 'Apr.', 'April.'] else x)
    date['month']=date['month'].apply(lambda x: '05' if x in ['May', 'May.'] else x)
    date['month']=date['month'].apply(lambda x: '06' if x in ['Jun', 'June', 'Jun.', 'June.'] else x)
    date['month']=date['month'].apply(lambda x: '07' if x in ['Jul', 'July', 'Jul.', 'July.'] else x)
    date['month']=date['month'].apply(lambda x: '08' if x in ['Aug', 'August', 'Aug.', 'August.'] else x)
    date['month']=date['month'].apply(lambda x: '09' if x in ['Sep', 'September', 'Sep.', 'September.'] else x)
    date['month']=date['month'].apply(lambda x: '10' if x in ['Oct', 'October', 'Oct.', 'October.'] else x)
    date['month']=date['month'].apply(lambda x: '11' if x in ['Nov', 'November', 'Nov.', 'November.'] else x)
    date['month']=date['month'].apply(lambda x: '12' if x in ['Dec', 'December', 'Decemeber', 'Dec.', 'December.'] else x)
    date['day']=date['day'].apply(lambda x: '0'+x if len(x)<=1 else x)
    date['date'] =pd.to_datetime(date['month'].apply(str)+'/'+date['day'].apply(str)+'/'+date['year'].apply(str))
    date.sort_values('date', inplace = True)
    date.reset_index(inplace = True)
    ans = date['level_0']
    return ans


date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    286
477    480
478    244
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64